In [1]:
from langchain.agents import create_agent
from langchain_core.tools import tool
from langchain.agents.middleware import HumanInTheLoopMiddleware

from langgraph.checkpoint.memory import InMemorySaver 

from typing_extensions import TypedDict

from dotenv import load_dotenv
load_dotenv()

class EmailRequest(TypedDict):
    recipient: str
    subject: str
    body: str


In [3]:
@tool()
def send_email(recipient: str, subject: str, body: str) -> str:
    """Send an email to a recipient.
    Args:
        recipient: Email address of the recipient.
        subject: Subject line of the email.
        body: Body content of the email.
    Returns:
        Confirmation message.
    """
    return f"Email sent successfully to {recipient}"


agent = create_agent(
    model="openai:gpt-4o",
    tools=[send_email],
    system_prompt="You are a helpful assistant for Sydney that can send emails.",
    middleware=[HumanInTheLoopMiddleware(interrupt_on={"send_email": True})],
    response_format=EmailRequest,
    checkpointer=InMemorySaver(), # In production, use a persistent checkpointer like AsyncPostgresSaver.
)

In [4]:
# Choose decision
# Approval decision
approval = {
    "decisions": [
        {
            "type": "approve"
        }
    ]
}

# Edit decision
edit = {
    "decisions": [
        {
            "type": "edit",
            "edited_action": {
                "name": "send_email",
                "args": {
                    "recipient": "partner@startup.com",
                    "subject": "Budget proposal for Q1 2026",
                    "body": "I can only approve up to 500k, please send over details.",
                }
            }
        }
    ]
}

# Reject decision
reject = {
    "decisions": [
        {
            "type": "reject",
            "message": "Please edit the email asking for more details about the budget proposal, then send the email"
        }
    ]
}

decision = edit  # Change this to approval or reject to test other flows

In [5]:
import uuid
config = {"configurable": {"thread_id": str(uuid.uuid4())}}

result = agent.invoke(
    {"messages": [{"role": "user", "content": "Send email to alice@example.com with subject Meeting and body Let's meet at 10 AM tomorrow."}]},
    config=config
)

print(result["__interrupt__"])

from langgraph.types import Command
if "__interrupt__" in result:
    description = result['__interrupt__'][-1].value['action_requests'][-1]['description']
    print("Human intervention required:", description)

    result = agent.invoke(
        Command(
            resume=decision
        ),
        config=config,    
    )

print("Final result:", result)

[Interrupt(value={'action_requests': [{'name': 'send_email', 'args': {'recipient': 'alice@example.com', 'subject': 'Meeting', 'body': "Let's meet at 10 AM tomorrow."}, 'description': 'Tool execution requires approval\n\nTool: send_email\nArgs: {\'recipient\': \'alice@example.com\', \'subject\': \'Meeting\', \'body\': "Let\'s meet at 10 AM tomorrow."}'}], 'review_configs': [{'action_name': 'send_email', 'allowed_decisions': ['approve', 'edit', 'reject']}]}, id='c05bebb46721253b95b0785d071747f7')]
Human intervention required: Tool execution requires approval

Tool: send_email
Args: {'recipient': 'alice@example.com', 'subject': 'Meeting', 'body': "Let's meet at 10 AM tomorrow."}
Final result: {'messages': [HumanMessage(content="Send email to alice@example.com with subject Meeting and body Let's meet at 10 AM tomorrow.", additional_kwargs={}, response_metadata={}, id='73d588bf-3034-4655-98e7-81a9eaa461f6'), AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'toke